TSG048 - Deployment stuck at “Waiting for controller pod to be up”
==================================================================

Description
-----------

Troubleshooting for the situation where running azdata create cluster
hangs at:

    Waiting for controller pod to be up…

Steps
-----

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream
except ImportError: 

    # Install the Kubernetes module
    import sys
    !{sys.executable} -m pip install kubernetes    
    
    try:
        from kubernetes import client, config
        from kubernetes.stream import stream
    except ImportError:
        display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
        raise

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    config.load_incluster_config()
else:
    try:
        config.load_kube_config()
    except:
        display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
        raise

api = client.CoreV1Api()

print('Kubernetes client instantiated')

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Get the name of controller pod that has the “couldn’t parse image reference problem”

In [ ]:
label_selector = 'app=controller'
name=api.list_namespaced_pod(namespace, label_selector=label_selector).items[0].metadata.name

print ("Controller pod name: " + name)

### Set the text to look for in pod events

In [ ]:
kind="Pod"
precondition_text="couldn't parse image reference"

### Get events for a Kubernetes resources

Get the events for a Kubernetes named space resource:

In [ ]:
V1EventList=api.list_namespaced_event(namespace)

for event in V1EventList.items:
    if (event.involved_object.kind==kind and event.involved_object.name==name):
        print(event.message)

### PRECONDITION CHECK

In [ ]:
precondition=False

for event in V1EventList.items:
    if (event.involved_object.kind==kind and event.involved_object.name==name):
        if event.message.find(precondition_text) != -1:
            precondition=True

if not precondition:
    raise Exception("PRECONDITION NON-MATCH: 'tsg048-create-stuck-waiting-for-controller' is not a match for an active problem")

print("PRECONDITION MATCH: 'tsg048-create-stuck-waiting-for-controller' is a match for an active problem in this cluster")

Resolution
----------

To resolve this problem fix the docker repository name and run cluster
create again.

In [ ]:
print("Notebook execution is complete.")